# 常见场景

## 意图识别

In [1]:
deployment = "gpt-4o"
model = "gpt-4o"

In [2]:

import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    # api_version="2023-12-01-preview", # 这个版本无 404 问题
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)


def chat(message: str) -> ChatCompletion:
    return client.chat.completions.create(
        model=deployment,
        messages=message,
        temperature=0.8,
        max_tokens=1000
    )


messages = [
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "提醒我明早8点有会议"},
    ],
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "订明天早5点北京到上海的飞机"},
    ]
]

for msg in messages:
    res = chat(msg)
    print(res.choices[0].message.content, res.usage.total_tokens)


好的，我会提醒你明早8点有会议。 37
好的，您要预订明天早上5点从北京到上海的飞机。我会帮您查询相关的航班信息并进行预订。 61


# 生成 SQL

In [4]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can answer the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)


为了计算所有学生英语课程的平均成绩，你需要查询两个表格并进行关联。首先，查找英语课程的ID，然后计算该课程中所有学生的平均成绩。

假设表结构如下：
- `students` 表：
  - `id`: 学生ID
  - `name`: 学生姓名
  - `course_id`: 课程ID
  - `score`: 成绩

- `courses` 表：
  - `id`: 课程ID
  - `name`: 课程名称

可以使用以下SQL查询来计算英语课程学生的平均成绩：

```sql
SELECT AVG(s.score) AS average_score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = '英语';
```

这个查询首先将 `students` 表和 `courses` 表通过 `course_id` 和 `id` 进行连接，然后过滤出课程名称为 "英语" 的记录，最后计算这些记录的平均成绩。


# 生成代码友好的提示词

In [5]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

sys_prompt = """Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "parameters" """

response = client.chat.completions.create(
    model=deployment,
    temperature=0.8,
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ],
)

print(response.choices[0].message.content, response.usage.total_tokens) 

```json
{
  "intention": "set_reminder",
  "parameters": {
    "time": "8:00 AM",
    "event": "meeting"
  }
}
``` 90


In [6]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can write a SQL string as the answer to the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)

```sql
SELECT AVG(s.score) AS average_score
FROM students s
JOIN courses c
ON s.course_id = c.id
WHERE c.name = '英语';
```


# 文本规范异常输出的格式

In [9]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can write a SQL string as the answer to the user request based on the given tables:
1. table “students“ with the columns [id, name, course_id, score] 
2. table "courses" with the columns [id, name] 
when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request" 
                  """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        # {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)

invalid request
